# Data Collection

In [1]:
# Retreive the Victim Dataset from Toronto Open Data API

import requests

base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
url = base_url + "/api/3/action/package_show"
params = { "id": "police-annual-statistical-report-victims-of-crime"}
package = requests.get(url, params = params).json()


# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

    # For datastore_active resources:
    if resource["datastore_active"]:

        # To get all records in CSV format:
        url = base_url + "/datastore/dump/" + resource["id"]
        try:
            resource_dump_data = requests.get(url).text
            # Success message
            print("Data retrieved successfully.")
        except requests.exceptions.RequestException as e:
            # Error message
            print(f"Error retrieving data: {e}")        

Data retrieved successfully.


In [2]:
# Create pandas Dataframe

import pandas as pd
from io import StringIO

# read csv file into pandas dataframe
csv_file = StringIO(resource_dump_data)
data = pd.read_csv(csv_file)

# Print the DataFrame
data.head()

,_id,REPORT_YEAR,CATEGORY,SUBTYPE,ASSAULT_SUBTYPE,SEX,AGE_GROUP,AGE_COHORT,COUNT_
0,1,2014,Crimes Against the Person,Sexual Violation,NaN,M,Adult,18 to 24,23
1,2,2014,Crimes Against the Person,Sexual Violation,NaN,M,Adult,25 to 34,29
2,3,2014,Crimes Against the Person,Sexual Violation,NaN,M,Adult,35 to 44,9
3,4,2014,Crimes Against the Person,Sexual Violation,NaN,M,Adult,45 to 54,13
4,5,2014,Crimes Against the Person,Sexual Violation,NaN,M,Adult,55 to 64,7


# Exploratory data analysis (EDA)

In [3]:
# Retrieve list of columns, their data types, and information about missing values.
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110 entries, 0 to 1109
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   _id              1110 non-null   int64 
 1   REPORT_YEAR      1110 non-null   int64 
 2   CATEGORY         1110 non-null   object
 3   SUBTYPE          1110 non-null   object
 4   ASSAULT_SUBTYPE  549 non-null    object
 5   SEX              1110 non-null   object
 6   AGE_GROUP        1110 non-null   object
 7   AGE_COHORT       1110 non-null   object
 8   COUNT_           1110 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 78.2+ KB


In [4]:
# Summary of statistics for the DataFrame columns.
data.describe()

,_id,REPORT_YEAR,COUNT_
count,1110.000000,1110.000000,1110.000000
mean,555.500000,2018.133333,218.891892
std,320.573704,2.554768,392.855422
min,1.000000,2014.000000,1.000000
25%,278.250000,2016.000000,5.250000
50%,555.500000,2018.000000,57.000000
75%,832.750000,2020.000000,247.000000
max,1110.000000,2022.000000,2528.000000


In [5]:
# Print the number of rows and columns
print("the dimension:", data.shape)

the dimension: (1110, 9)


In [6]:
# Selects the values of the first row of DataFrame
data_values = data.iloc[0]
data_values

_id                                        1
REPORT_YEAR                             2014
CATEGORY           Crimes Against the Person
SUBTYPE                     Sexual Violation
ASSAULT_SUBTYPE                          NaN
SEX                                        M
AGE_GROUP                              Adult
AGE_COHORT                          18 to 24
COUNT_                                    23
Name: 0, dtype: object

In [7]:
# Print unique values for categorical values
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    unique_count = data[col].nunique()
    print("Unique count of column '{}': {}".format(col, unique_count))

Unique count of column 'CATEGORY': 1
Unique count of column 'SUBTYPE': 4
Unique count of column 'ASSAULT_SUBTYPE': 5
Unique count of column 'SEX': 3
Unique count of column 'AGE_GROUP': 4
Unique count of column 'AGE_COHORT': 9


###### We can observe that this dataset is aggregated; thus, minor adjustments will be necessary.

In [8]:
# Show unique values of the 'REPORT_YEAR' column
data['REPORT_YEAR'].value_counts().sort_index()

REPORT_YEAR
2014    109
2015    113
2016    124
2017    122
2018    124
2019    131
2020    129
2021    128
2022    130
Name: count, dtype: int64

# Data Cleaning and Transformation

In [9]:
# Get count and percentage of Null Values

data1 = data.copy()

data_null_count = data1.isnull().sum()
data_null_percentage = (data1.isnull().sum() / data1.shape[0]) * 100
null_values = pd.DataFrame({
    'Null Count': data_null_count,
    'Null Percentage': data_null_percentage
})
null_values

,Null Count,Null Percentage
_id,0,0.000000
REPORT_YEAR,0,0.000000
CATEGORY,0,0.000000
SUBTYPE,0,0.000000
ASSAULT_SUBTYPE,561,50.540541
SEX,0,0.000000
AGE_GROUP,0,0.000000
AGE_COHORT,0,0.000000
COUNT_,0,0.000000


In [14]:
# Identify potential cause of null values in 'ASSAULT_SUBTYPE' column.

grouped_counts = data1.groupby('SUBTYPE')['ASSAULT_SUBTYPE'].agg(count_assault='count').reset_index()

grouped_counts


,SUBTYPE,count_assault
0,Assault,549
1,Other,0
2,Robbery,0
3,Sexual Violation,0


###### We chose to Keep the null values in the dataset, as they exclusively represent subtypes other than 'Assault' in the 'ASSAULT_SUBTYPE' column.

In [15]:
# Save the DataFrame to an Excel file for use in Power BI for visualization
data1.to_excel('../Data/TOR_victims.xlsx', index=False) 